In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc


/home/jaromir/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
##### READ IN RAW DATA

print( "\nReading data from disk ...")
prop = pd.read_csv('Data/properties_2016.csv')
train = pd.read_csv("Data/train_2016_v2.csv")



Reading data from disk ...


/home/jaromir/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
prop.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [12]:
##### PROCESS DATA FOR LIGHTGBM

print( "\nProcessing data for LightGBM ..." )
for c, dtype in zip(prop.columns, prop.dtypes):	
    if dtype == np.float64:		
        prop[c] = prop[c].astype(np.float32)



Processing data for LightGBM ...


In [13]:
train.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [14]:
df_train = train.merge(prop, how='left', on='parcelid')
df_train.fillna(df_train.median(),inplace = True)


,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,7.0,616.0,2.0,3.0,4.0,4.0,...,1.0,1,122754.0,360170.0,2015.0,237416.0,6735.879883,NaN,14.0,6.037107e+13
1,14366692,-0.1684,2016-01-01,1.0,7.0,616.0,3.5,4.0,4.0,7.0,...,1.0,1,346458.0,585529.0,2015.0,239071.0,10153.019531,NaN,14.0,6.037615e+13
2,12098116,-0.0040,2016-01-01,1.0,7.0,616.0,3.0,2.0,4.0,4.0,...,1.0,1,61994.0,119906.0,2015.0,57912.0,11484.480469,NaN,14.0,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,7.0,616.0,2.0,2.0,4.0,4.0,...,1.0,1,171518.0,244880.0,2015.0,73362.0,3048.739990,NaN,14.0,6.037296e+13
4,14432541,-0.0050,2016-01-02,1.0,7.0,616.0,2.5,4.0,4.0,7.0,...,2.0,1,169574.0,434551.0,2015.0,264977.0,5488.959961,NaN,14.0,6.059042e+13
5,11509835,-0.2705,2016-01-02,1.0,7.0,616.0,4.0,4.0,4.0,1.0,...,1.0,1,880650.0,2447951.0,2015.0,1567301.0,27126.570312,NaN,14.0,6.037621e+13
6,12286022,0.0440,2016-01-02,1.0,7.0,616.0,1.0,2.0,4.0,7.0,...,1.0,1,64549.0,111521.0,2015.0,46972.0,2304.969971,NaN,14.0,6.037542e+13
7,17177301,0.1638,2016-01-02,1.0,7.0,616.0,2.5,3.0,4.0,7.0,...,2.0,1,107000.0,306000.0,2015.0,199000.0,3745.500000,NaN,14.0,6.111003e+13
8,14739064,-0.0030,2016-01-02,1.0,7.0,616.0,1.0,2.0,4.0,7.0,...,1.0,1,66834.0,210064.0,2015.0,143230.0,2172.879883,NaN,14.0,6.059042e+13
9,14677559,0.0843,2016-01-03,1.0,7.0,616.0,2.0,2.0,4.0,7.0,...,1.0,1,109977.0,190960.0,2015.0,80983.0,1940.260010,NaN,14.0,6.059063e+13


In [15]:
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 
                         'propertycountylandusecode', 'fireplacecnt', 'fireplaceflag'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)



(90275, 53) (90275,)


In [20]:
x_train.dtypes[x_train.dtypes == object].index.values

array(['hashottuborspa', 'taxdelinquencyflag'], dtype=object)

In [23]:
train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

In [25]:
del df_train; gc.collect()

3467

In [26]:
x_train = x_train.values.astype(np.float32, copy=False)
d_train = lgb.Dataset(x_train, label=y_train)


In [27]:
params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.0021 # shrinkage_rate
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l1'          # or 'mae'
params['sub_feature'] = 0.5      # feature_fraction -- OK, back to .5, but maybe later increase this
params['bagging_fraction'] = 0.85 # sub_row
params['bagging_freq'] = 40
params['num_leaves'] = 512        # num_leaf
params['min_data'] = 500         # min_data_in_leaf
params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf
params['verbose'] = 0

In [28]:
print("\nFitting LightGBM model ...")
clf = lgb.train(params, d_train, 430)



Fitting LightGBM model ...


In [29]:
print("\nPrepare for LightGBM prediction ...")
print("   Read sample file ...")
sample = pd.read_csv('Data/sample_submission.csv')
print("   ...")
sample['parcelid'] = sample['ParcelId']
print("   Merge with property data ...")
df_test = sample.merge(prop, on='parcelid', how='left')
print("   ...")
del sample, prop; gc.collect()
print("   ...")


Prepare for LightGBM prediction ...
   Read sample file ...
   ...
   Merge with property data ...
   ...
   ...


In [30]:
train_columns

Index(['airconditioningtypeid', 'architecturalstyletypeid', 'basementsqft',
       'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertylandusetypeid', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt',
       'numberofstories', 'structuretaxvaluedollarcnt', 'taxval

In [31]:
x_test = df_test[train_columns]
print("   ...")
del df_test; gc.collect()
print("   Preparing x_test...")
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
print("   ...")
x_test = x_test.values.astype(np.float32, copy=False)

   ...
   Preparing x_test...
   ...


In [32]:
print("\nStart LightGBM prediction ...")
# num_threads > 1 will predict very slow in kernal
clf.reset_parameter({"num_threads":1})
p_test = clf.predict(x_test)



Start LightGBM prediction ...


In [33]:
del x_test; gc.collect()

print( "\nUnadjusted LightGBM predictions:" )
print( pd.DataFrame(p_test).head() )



Unadjusted LightGBM predictions:
          0
0  0.042084
1  0.045119
2  0.020453
3  0.016081
4  0.009081
